In [10]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input
from collections import OrderedDict

In [2]:
image_folder = "all_images"
image_list = []
label_list = []
pfirrman_grades = []
modic_types = []
df = pd.read_csv('AggregatePTData.csv')

In [3]:
for image_filename in os.listdir(image_folder):
    if image_filename.endswith(".png"):
        image = Image.open(os.path.join(image_folder, image_filename))
        # Resize image to 224x224 pixels
        image = image.resize((224,224))
        # Convert image to RGB
        image = image.convert("RGB")
        # Convert the image to numpy array
        image_array = np.array(image)
        # Normalize array
        image_array = preprocess_input(image_array)
        image_list.append(image_array)
        
        row = df[df['file_name'] == image_filename]
        pfirrman_grade = row.iloc[0]['Pfirrman grade']
        pfirrman_grades.append(pfirrman_grade)
        modic = row.iloc[0]['Modic']
        modic_types.append(modic)

image_list_np = np.array(image_list)
pfirrman_grades_np = np.array(pfirrman_grades)
pfirrman_grades_np = np.round(pfirrman_grades_np).astype(int) - 1
modic_types_np = np.array(modic_types)
modic_types_np = np.round(modic_types_np).astype(int)

In [5]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Fully connected layer
pfirrman_predictions = Dense(5, activation='softmax', name='pfirrman_output')(x)
modic_predictions = Dense(3, activation='softmax', name='modic_output')(x)

In [6]:
model = Model(inputs=base_model.input, outputs=[pfirrman_predictions, modic_predictions])
model.compile(optimizer='adam', loss={
        'pfirrman_output': 'sparse_categorical_crossentropy', 
        'modic_output': 'sparse_categorical_crossentropy'
    },          
    metrics={
        'pfirrman_output': 'accuracy',
        'modic_output': 'accuracy'
    }
)

In [13]:
history = model.fit(
    image_list_np,            # Input images
    [pfirrman_grades_np, modic_types_np],
    batch_size=32,
    epochs=10,
    validation_split=0.2,     # Split 20% of data for validation
    verbose=1                 # Set to 1 to see training progress
)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - loss: 4.9532 - modic_output_accuracy: 0.5283 - modic_output_loss: 1.7153 - pfirrman_output_accuracy: 0.2160 - pfirrman_output_loss: 3.2379 - val_loss: 3.8203 - val_modic_output_accuracy: 0.4795 - val_modic_output_loss: 1.9256 - val_pfirrman_output_accuracy: 0.3973 - val_pfirrman_output_loss: 1.9222
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 2.7674 - modic_output_accuracy: 0.5421 - modic_output_loss: 1.1917 - pfirrman_output_accuracy: 0.5010 - pfirrman_output_loss: 1.5757 - val_loss: 2.5535 - val_modic_output_accuracy: 0.4521 - val_modic_output_loss: 1.1062 - val_pfirrman_output_accuracy: 0.3973 - val_pfirrman_output_loss: 1.5164
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 1.6683 - modic_output_accuracy: 0.7097 - modic_output_loss: 0.6806 - pfirrman_output_accuracy: 0.5936 - pfirrman_output_loss: 0.9877 - val_loss: 2.5987 - val_modic_output_accuracy: 0.5342 - val_modic_output_loss: 1.0127 - val_pfirrman_output_ac

In [30]:
test_image_path = 'all_images/51_t1.png'

# Load, resize, and preprocess the test image
img = Image.open(test_image_path).convert('RGB')
img_resized = img.resize((224, 224))
img_array = np.array(img_resized)

# Preprocess the image for ResNet50
img_preprocessed = tf.keras.applications.resnet50.preprocess_input(img_array)
img_preprocessed = np.expand_dims(img_preprocessed, axis=0)  # Add batch dimension

# Run the prediction
predicted_probabilities = model.predict(img_preprocessed)
predicted_pfirrman = np.argmax(predicted_probabilities[0]) + 1  # Add 1 if classes are 1-5
predicted_modic = np.argmax(predicted_probabilities[1])
print(f"Predicted Pfirrman grade: {predicted_pfirrman}")
print(f"Predicted Modic: {predicted_modic}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Pfirrman grade: 3
Predicted Modic: 0


In [ ]:
model.summary()

In [31]:
model.save('nonBinaryPredictions.keras')